In [1]:
import qiskit
qiskit.__qiskit_version__

{'qiskit-terra': '0.17.1', 'qiskit-aer': '0.8.1', 'qiskit-ignis': '0.6.0', 'qiskit-ibmq-provider': '0.12.2', 'qiskit-aqua': '0.9.1', 'qiskit': '0.25.1', 'qiskit-nature': None, 'qiskit-finance': None, 'qiskit-optimization': None, 'qiskit-machine-learning': None}

In [2]:
import numpy as np
import matplotlib.pyplot as plt

from torch import Tensor
from torch.nn import MSELoss
from torch.optim import LBFGS, SGD,Adam 

from qiskit  import Aer, QuantumCircuit
from qiskit.utils import QuantumInstance
from qiskit.circuit import Parameter
from qiskit_machine_learning.neural_networks import CircuitQNN, TwoLayerQNN
from qiskit_machine_learning.connectors import TorchConnector

import torch.optim as optim

from pandas.core.common import flatten

# Load in some additional functions from a text file
from helpers import parity

import sys
sys.path.append('../../Pyfiles')
from circuits_n1 import *

In [3]:
# Set up the QuantumInstance we'll use to run things
qi = QuantumInstance(Aer.get_backend('statevector_simulator'))

In [4]:
import torch
torch.cuda.is_available()

True

# Pull in a data set

In [5]:
# Replace this
dataSetID = '0'

# Define the directory paths
data0Path = r'../../dataset/data{0}.txt'.format(dataSetID)
data0Label = r'../../dataset/data{0}label.txt'.format(dataSetID)

# Load in the data
dataCoords = np.loadtxt(data0Path)
dataLabels = np.loadtxt(data0Label)

# Make a data structure which is easier to work with
# for shuffling. 
# Also, notice we change the data labels from {0, 1} to {-1, +1}
data = list(zip(dataCoords, 2*dataLabels-1))

In [6]:
def generate_train_validate_test_data(data, train_size=100, validate_size=500, randomSeed=0):
    r'''This is a function which, given a dataset, will return 3 distinct datasets from it: a training dataset,
    a validation dataset, and a testing dataset.
    
    The size of the training and validation datasets is set by the function call; the size of the testing
    dataset is the remainder of the data after training & validation dataset have been generated.'''
    
    assert len(data) > train_size+validate_size, 'Not enough data to do the splitting.'
    
    def generate_data(data, ixs):
        r'''Helper function for generating data.'''
        X= [np.array(list(flatten([data[j][0],data[j][0]]))) for j in ixs]
        y = [data[j][1] for j in ixs]    
    
        # Recast X as a pyTorch tensor
        X = Tensor(X)
        
        # Change how the data is labeled: {-1,+1} --> {0, 1}
        y =  [ (x + 1)/2 for x in y]
        
        return X, y
    
    # At the start, we could use all possible datapoints for training
    possible_ixs = range(len(data))
    
    # Training data
    np.random.seed(randomSeed)
    train_ixs = np.random.choice(possible_ixs, size=train_size)
    train_X, train_y = generate_data(data, train_ixs)
    
    # Update the possible indices we could choose from
    possible_ixs = [x for x in possible_ixs if x not in train_ixs]

    # Validation data
    np.random.seed(randomSeed)
    validate_ixs = np.random.choice(possible_ixs, size=validate_size)
    validate_X, validate_y = generate_data(data, validate_ixs)
    
    # Now, use the rest of the data for testing
    possible_ixs = [x for x in possible_ixs if x not in validate_ixs]
    test_X, test_y = generate_data(data, possible_ixs)
    
    return train_X, train_y, validate_X, validate_y, test_X, test_y

In [7]:
train_X, train_y, validate_X, validate_y, test_X, test_y = generate_train_validate_test_data(data, train_size=100, validate_size=500)

# Make the circuit

In [8]:
# Make the feature map
feature_map = QuantumCircuit(4, name='Embed')
feature_map.rx(Parameter('x[0]'),0)
feature_map.rx(Parameter('x[1]'),1)
feature_map.rx(Parameter('x[2]'),2)
feature_map.rx(Parameter('x[3]'),3)

for j in [0, 2]:
    feature_map.ry(np.pi/4,j)
    feature_map.ry(np.pi/4,j+1)
    feature_map.rz(np.pi/4,j)
    feature_map.rz(np.pi/4,j+1)

# Make an ansazt
param_y=[(Parameter('θ'+str(i))) for i in range(12)]
ansatz = QuantumCircuit(4, name='PQC')
ansatz=circuit1(ansatz,param_y,1,0)


qc = QuantumCircuit(ansatz.width())
qc.append(feature_map, range(ansatz.width()))
qc.append(ansatz, range(ansatz.width()))

qc.draw()

┌─────────────────────────────┐┌───────────────────────────────┐
q_0: ┤0                            ├┤0                              ├
     │                             ││                               │
q_1: ┤1                            ├┤1                              ├
     │  Embed(x[0],x[1],x[2],x[3]) ││  PQC(θ0,θ1,θ2,θ3,θ4,θ5,θ6,θ7) │
q_2: ┤2                            ├┤2                              ├
     │                             ││                               │
q_3: ┤3                            ├┤3                              ├
     └─────────────────────────────┘└───────────────────────────────┘

# Train the network

In [9]:
def check_accuracy(model, X, y_target):
    r'''Helper function to compute the accuracy'''
    
    # Evaluate model on input data
    output = model(X)
    
    # Now, do some wrangling to get the data in a better format
    output = output.detach().numpy()
    
    # Output is a list of lists, where the inner list is the probabilities
    # of each class assignment. We pick the most probable class as the prediction
    predictions = np.array([np.argmax(x) for x in output])
    
    return sum(predictions == y_target)/len(y_target)

In [33]:
learning_rate = .5
epochs = 20
output_shape = 2

In [ ]:
learning_rate=[round(((i+1)/10)**2,2) for i in range(20)]
storage_model=[]
for l in learning_rate: 
    # Create the QNN:
    qnn = CircuitQNN(qc, input_params=feature_map.parameters, weight_params=ansatz.parameters, 
                      interpret=parity, output_shape=output_shape, quantum_instance=qi)

    # Set up the pyTorch model
    np.random.seed(0)  
    initial_weights = 0.1*(2*np.random.rand(qnn.num_weights) - 1)
    model = TorchConnector(qnn, initial_weights)

    # define optimizer and loss function
    optimizer = optim.SGD(model.parameters(),lr=l)
    f_loss = MSELoss(reduction='mean')

    # Set model to training mode
    model.train()   

    print("Learning Rate is ", l)
    for epoch in range(epochs):
        optimizer.zero_grad()        # initialize gradient
        loss = 0.0                                             # initialize loss    
        for x, y_target in zip(train_X, train_y):                        # evaluate batch loss
            output = model(Tensor(x)).reshape(1, 2)           # forward pass
            targets=Tensor([y_target]).long()
            targets = targets.to(torch.float32)
            loss += f_loss(output, targets) 
        loss.backward()                              # backward pass
        print("__Loss is ",loss.item())                           # print loss

        # run optimizer
        optimizer.step() 
    # save accuracy and parameters
    output_train_X = model(train_X)
    output_validate_X = model(validate_X)
    output_test_X = model(test_X)
    #Loading is taking most of the time.^^
    predictions_train_X = np.array([np.argmax(x) for x in output_train_X.detach().numpy()])    
    predictions_validate_X = np.array([np.argmax(x) for x in output_validate_X.detach().numpy()])
    predictions_test_X = np.array([np.argmax(x) for x in output_test_X.detach().numpy()])
    accuracy_train = np.round(sum(predictions_train_X == train_y)/len(train_y), 4)
    accuracy_validate = np.round(sum(predictions_validate_X == validate_y)/len(validate_y), 4)
    accuracy_test = np.round(sum(predictions_test_X == test_y)/len(test_y), 4)
    print("Accuracies are",accuracy_train,accuracy_validate,accuracy_validate)
    param_model=[p.data for p in model.parameters()]
    storage_model.append([l,param_model,accuracy_train,accuracy_validate,accuracy_test])

Learning Rate is  0.01
__Loss is  28.43077850341797
__Loss is  28.395334243774414
__Loss is  28.35608673095703
__Loss is  28.31281089782715
__Loss is  28.26523780822754
__Loss is  28.213159561157227
__Loss is  28.156373977661133
__Loss is  28.09478187561035
__Loss is  28.02828598022461
__Loss is  27.956897735595703
__Loss is  27.880706787109375
__Loss is  27.799880981445312
__Loss is  27.714706420898438
__Loss is  27.62554359436035
__Loss is  27.532869338989258
__Loss is  27.43724250793457
__Loss is  27.339303970336914
__Loss is  27.239728927612305
__Loss is  27.139244079589844
__Loss is  27.03860855102539
Accuracies are 0.49 0.472 0.472
Learning Rate is  0.04
__Loss is  28.43077850341797
__Loss is  28.27850341796875
__Loss is  28.06242561340332
__Loss is  27.773792266845703
__Loss is  27.418380737304688
__Loss is  27.02275848388672
__Loss is  26.629058837890625
__Loss is  26.277498245239258
__Loss is  25.99020767211914
__Loss is  25.769092559814453
__Loss is  25.60418701171875
__Loss 

In [ ]:
storage_model

# Evaluate the accuracy of the trained network

We'll make plots of the different data sets, and circle which data points the trained model gets wrong.
We'll also title each plot with the value of the accuracy of the model on that data set.

In [ ]:
def make_accuracy_plot(X_data, y_target, model, ax):
    output = model(X_data)
    predictions = np.array([np.argmax(x) for x in output.detach().numpy()])
    for x, y_tar, y_pred in zip(X_data, y_target, predictions):
        if y_tar == 1:
            ax.plot(x[0], x[1], 'bo')
        else:
            ax.plot(x[0], x[1], 'go')
        if y_tar != y_pred:
            ax.scatter(x[0], x[1], s=200, facecolors='none', edgecolors='y', linewidths=2)


    X1 = np.linspace(0, 1, num=10)
    Z1 = np.zeros((len(X1), len(X1)))

    # Contour map
    for j in range(len(X1)):
        for k in range(len(X1)):
            # Fill Z with the labels (numerical values)
            # the inner loop goes over the columns of Z,
            # which corresponds to sweeping x-values
            # Therefore, the role of j,k is flipped in the signature
            Z1[j, k] = np.argmax(model(Tensor([X1[k],X1[j],X1[k],X1[j]])).detach().numpy())

    ax.contourf(X1, X1, Z1, levels=30, zorder=-1, cmap='bwr')

    accuracy = np.round(sum(predictions == y_target)/len(y_target), 4)

    return ax, accuracy

In [ ]:
fig = plt.figure(figsize=(15, 5))

ax1 = fig.add_subplot(1, 3, 1)
ax2 = fig.add_subplot(1, 3, 2)
ax3 = fig.add_subplot(1, 3, 3)

ax1, accuracy1 = make_accuracy_plot(train_X, train_y, model, ax1)
ax2, accuracy2 = make_accuracy_plot(validate_X, validate_y, model, ax2)
ax3, accuracy3 = make_accuracy_plot(test_X, test_y, model, ax3)

ax1.set_title('Train: {0}'.format(accuracy1))
ax2.set_title('Validate: {0}'.format(accuracy2))
ax3.set_title('Test: {0}'.format(accuracy3))

In [35]:
print(1)
output_train_X = model(train_X)
print(1)
output_validate_X = model(validate_X)
print(1)
output_test_X = model(test_X)
print(1)
predictions_train_X = np.array([np.argmax(x) for x in output_train_X.detach().numpy()])    
print(2)
predictions_validate_X = np.array([np.argmax(x) for x in output_validate_X.detach().numpy()])
print(2)
predictions_test_X = np.array([np.argmax(x) for x in output_test_X.detach().numpy()])
print(2)
accuracy_train = np.round(sum(predictions_train_X == train_y)/len(train_y), 4)
print(3)
accuracy_validate = np.round(sum(predictions_validate_X == validate_y)/len(validate_y), 4)
print(3)
accuracy_test = np.round(sum(predictions_test_X == test_y)/len(test_y), 4)
print(3)
print("Accuracies are",accuracy_train,accuracy_validate,accuracy_validate)
param_model=[p.data for p in model.parameters()]
print(4)
storage_model.append([l,param_model,accuracy_train,accuracy_validate,accuracy_test])

1
1
1
1
2
2
2
3
3
3
Accuracies are 0.49 0.472 0.472
4
